In [1]:
import pandas as pd
import numpy as np

from itertools import tee, izip
import os

In [2]:
train = pd.read_csv('/home/ec2-user/Kaggle/facebook_Jul_2016/feature_engineered_input/hr_wk_mth/train_hr_wk_mth.csv')
test = pd.read_csv('/home/ec2-user/Kaggle/facebook_Jul_2016/feature_engineered_input/hr_wk_mth/test_hr_wk_mth.csv')

# for real testing set we don't need to do this - it doesn't have place_id info
#test = test[['row_id', 'x', 'y', 'accuracy', 'time']]

print train.head()
print test.head()

   row_id       x       y  accuracy    time    place_id  hour  weekday  month
0       0  0.7941  9.0809        54  470702  8523065625    22        5     11
1       1  5.9567  4.7968        13  186555  1757726713    14        4      5
2       2  8.3078  7.0407        74  322648  1137537235     2        1      8
3       3  7.3665  2.5165        65  704587  6567393236     8        7      5
4       4  4.0961  1.1307        31  472130  7440663949    21        6     11
   row_id       x       y  accuracy    time  hour  weekday  month
0       0  0.1675  1.3608       107  930883    11        3     10
1       1  7.3909  2.5301        35  893017     4        5      9
2       2  8.0978  2.3473        62  976933    11        7     11
3       3  0.9990  1.0591        62  907285     2        1     10
4       4  0.6670  9.7254        40  914399    24        5     10


In [3]:
import K_fbV.fb_split_grid as sg
import K_fbV.factorize_predictor as fp

In [4]:
# generate the grids
train_grid, test_grid = sg.get_grids(train, test, n = 40, m = 40)

print test_grid[(2,8)].shape # number of testing set data points in grid (2, 8)

(5664, 8)


In [5]:
import K_fbV.feature_predictor_split as fps
import K_fbV.tune_numRound_maxDepth as tnm

import K_fbV.train_grid_model as tgm
import K_fbV.make_grid_prediction as mgp

import K_fbV.gen_submission as gs

In [6]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob' # tells boosted trees to output probability
param['booster'] = 'gbtree' # - default is set to "gbtree" - gradient boosted tree
# scale weight of positive examples
param['eta'] = 0.1 # Parameters for Tree Booster - Booster parameter
param['silent'] = 1 # whether to print logs
param['nthread'] = 36 # parallelism
param['eval_metric'] = 'mlogloss'

# Quick smoke test of the entire pipeline

In [7]:
model_dict = {}
unique_class_dict = {}

tgm.train_model((2, 8), train_grid, param, model_dict, unique_class_dict,
            feature_list = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month'],
            predictor_name = 'place_id', leaf_mltpr = 1.5, tree_mltpr = 1.5)

tgm.train_model((2, 9), train_grid, param, model_dict, unique_class_dict,
            feature_list = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month'],
            predictor_name = 'place_id', leaf_mltpr = 1.5, tree_mltpr = 1.5)

[0]	train-mlogloss:4.195870
[1]	train-mlogloss:3.484432
[2]	train-mlogloss:3.095367
[3]	train-mlogloss:2.860185
[4]	train-mlogloss:2.688941
[5]	train-mlogloss:2.546095
[6]	train-mlogloss:2.423606
[7]	train-mlogloss:2.315287
[8]	train-mlogloss:2.220860
[9]	train-mlogloss:2.137168
[10]	train-mlogloss:2.062099
[11]	train-mlogloss:1.992953
[12]	train-mlogloss:1.931711
[13]	train-mlogloss:1.872575
[14]	train-mlogloss:1.816527
[15]	train-mlogloss:1.764913
[16]	train-mlogloss:1.716289
[17]	train-mlogloss:1.671995
[18]	train-mlogloss:1.627555
[19]	train-mlogloss:1.587991
[20]	train-mlogloss:1.550178
[21]	train-mlogloss:1.514879
[22]	train-mlogloss:1.480698
[23]	train-mlogloss:1.448794
[24]	train-mlogloss:1.416662
[25]	train-mlogloss:1.386043
[26]	train-mlogloss:1.356963
[27]	train-mlogloss:1.329431
[28]	train-mlogloss:1.303879
[29]	train-mlogloss:1.277325
[30]	train-mlogloss:1.251948
[31]	train-mlogloss:1.227417
[32]	train-mlogloss:1.204593
[33]	train-mlogloss:1.182724
[34]	train-mlogloss:1.16

number of boosters in training: 37
number of boosters in training: 37

[36]	train-mlogloss:1.118550
[0]	train-mlogloss:4.547091
[1]	train-mlogloss:3.634774
[2]	train-mlogloss:4.403870
[3]	train-mlogloss:2.988979
[4]	train-mlogloss:2.673471
[5]	train-mlogloss:2.531318
[6]	train-mlogloss:2.412554
[7]	train-mlogloss:2.310858
[8]	train-mlogloss:2.221137
[9]	train-mlogloss:2.142741
[10]	train-mlogloss:2.072070
[11]	train-mlogloss:2.008550
[12]	train-mlogloss:1.949577
[13]	train-mlogloss:1.893833
[14]	train-mlogloss:1.843325
[15]	train-mlogloss:1.796056
[16]	train-mlogloss:1.752271
[17]	train-mlogloss:1.710870
[18]	train-mlogloss:1.672307
[19]	train-mlogloss:1.635218
[20]	train-mlogloss:1.600310
[21]	train-mlogloss:1.565492
[22]	train-mlogloss:1.533899
[23]	train-mlogloss:1.503062
[24]	train-mlogloss:1.474283
[25]	train-mlogloss:1.446629
[26]	train-mlogloss:1.418525
[27]	train-mlogloss:1.393007
[28]	train-mlogloss:1.368435
[29]	train-mlogloss:1.344418
[30]	train-mlogloss:1.320867
[31]	train-mlogloss:1.298674
[32]	train-mlogloss:1.277074
[33]	train-mlogloss:1.25

In [ ]:
prediction_dict = {}
mgp.predict_output((2, 8), test_grid, model_dict, unique_class_dict, prediction_dict,
               test_row_id = 'row_id', feature_list = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month'])

mgp.predict_output((2, 9), test_grid, model_dict, unique_class_dict, prediction_dict,
               test_row_id = 'row_id', feature_list = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month'])

print gs.merge_prediction_grids(prediction_dict).head()

# multi-process Parallalization of the above pipeline to use all cores
   - specify a pool of processes using multiprocess package
   - need wrapper a function for each function with input parameters more than 1
   - use itertools to zip arguments in a list, and in wrapper function unzip/unwrap it

In [ ]:
def parallel_train(inputs):
    """Convert f([1,2]) to f(1,2) call"""
    return tgm.train_model(*inputs)

In [ ]:
import multiprocessing
from multiprocessing import *
from multiprocessing import Queue, Process, freeze_support
from multiprocessing import Pool

import itertools # for multiple input arguments of the function to be parallilized

multiprocessing.cpu_count() # print the number of CPUs availalbe to system

freeze_support()
pool = Pool( processes = int(multiprocessing.cpu_count() * 0.1) ) # process pool based on number of cores 

# define input arguments with their associated names in function
feature_list = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month']
predictor_name = 'place_id'
leaf_mltpr = 1.002
tree_mltpr = 1.001

pool.map(parallel_train, # function to be parallilized
         itertools.izip
        (
            train_grid.keys(),
            itertools.repeat(train_grid),
            itertools.repeat(param),
            itertools.repeat(model_dict),
            itertools.repeat(unique_class_dict),
            itertools.repeat(feature_list),
            itertools.repeat(predictor_name),
            itertools.repeat(leaf_mltpr),
            itertools.repeat(tree_mltpr)
        )
    )
print model_dict